In [1]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

# Setup

In [2]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

'python'

In [3]:
# third party
from apis import make_submit_query
from apis import make_test_query
from helpers import create_user
from helpers import make_user

# syft absolute
import syft as sy
from syft import test_settings

Using Mock API Code, this will query BigQuery. $TEST_BIGQUERY_APIS_LIVE==False


In [4]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [5]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080
Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Will watch for changes in these directories: ['/home/teo/OpenMined/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [241047] using WatchFiles


Waiting for server to start..WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.
.

INFO:     Started server process [241061]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


.INFO:     127.0.0.1:54092 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:8080.It is intended only for local use.

In [6]:
high_client = sy.login(
    url=SERVER_URL, email="info@openmined.org", password="changethis"
)

INFO:     127.0.0.1:54108 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:54118 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [7]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

INFO:     127.0.0.1:54120 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:54130 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "GET /api/v2/api?verify_key=edf4433715069a623e77a08c75617167e103a54b606fcde79540bbec4ac75d0f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:54166 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <samantha-newton-fake@openmined.org>
INFO:     127.0.0.1:54172 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:54188 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:54190 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:54108 - "GET /api/v2/api?verify_key=09c59043551500477

In [8]:
mock_func = make_test_query(
    settings={
        "rate_limiter_enabled": True,
        "calls_per_min": 10,
    }
)

private_func = make_test_query(
    settings={
        "rate_limiter_enabled": False,
    }
)

new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=new_endpoint)

INFO:     127.0.0.1:54236 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

In [9]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_1_col_id = test_settings.get("table_1_col_id", default="table_id")
table_1_col_score = test_settings.get("table_1_col_score", default="colname")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [10]:
submit_query_function = make_submit_query(
    settings={},
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=submit_query_function)

INFO:     127.0.0.1:54246 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

# Create jobs

In [11]:
# third party
from job_helpers import TestJob
from job_helpers import create_jobs
from job_helpers import submit_job

In [12]:
n_per_user = 5

jobs = create_jobs(users, n_per_user=n_per_user)

print(f"num jobs: {len(jobs)}")
for job in jobs[:2]:
    print(f"Job type: {job.job_type}, should succeed: {job.should_succeed}")

assert len(jobs) == len(users) * n_per_user
assert all(isinstance(j, TestJob) for j in jobs)
assert all(job.client is not None for job in jobs)

num jobs: 10
Job type: job_query_xss, should succeed: False
Job type: job_funcname_xss, should succeed: True


In [13]:
responses = []

for job in jobs:
    try:
        responses.append(submit_job(job))
    except:
        print("WE FAILED ON:", job.func_name)

INFO:     127.0.0.1:54108 - "GET /api/v2/api?verify_key=edf4433715069a623e77a08c75617167e103a54b606fcde79540bbec4ac75d0f&communication_protocol=dev HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:54256 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/utils.py", line 63, in parse_code
    tree = ast.parse(raw_code)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.12/lib/python3.12/ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 2
    def job_funcname_xss_5df941<script>alert('XSS in funcname')</script>(query: str, endpoint):
                               ^
SyntaxError: expected '('

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1300, in decorator
    parse_user_code(
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1368, in parse_user_cod

SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
INFO:     127.0.0.1:47986 - "POST /api/v2/api_call HTTP/1.1" 200 OK
WE FAILED ON: job_funcname_xss_5df941<script>alert('XSS in funcname')</script>
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:48086 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:38838 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/utils.py", line 63, in parse_code
    tree = ast.parse(raw_code)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.12/lib/python3.12/ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 2
    def job_funcname_xss_c759bf<script>alert('XSS in funcname')</script>(query: str, endpoint):
                               ^
SyntaxError: expected '('

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1300, in decorator
    parse_user_code(
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1368, in parse_user_cod

SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
INFO:     127.0.0.1:46660 - "POST /api/v2/api_call HTTP/1.1" 200 OK
WE FAILED ON: job_funcname_xss_c759bf<script>alert('XSS in funcname')</script>
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:59778 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/utils.py", line 63, in parse_code
    tree = ast.parse(raw_code)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.12/lib/python3.12/ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 2
    def job_funcname_xss_f29303<script>alert('XSS in funcname')</script>(query: str, endpoint):
                               ^
SyntaxError: expected '('

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1300, in decorator
    parse_user_code(
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1368, in parse_user_cod

SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
INFO:     127.0.0.1:53214 - "POST /api/v2/api_call HTTP/1.1" 200 OK
WE FAILED ON: job_funcname_xss_f29303<script>alert('XSS in funcname')</script>
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:53362 - "POST /api/v2/api_call HTTP/1.1" 200 OK


In [14]:
assert all(job.is_submitted for job in jobs if not job.func_name.startswith("job_funcname_xss"))

## Test: cannot execute

In [15]:
# Blocking

for job in jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = execute_code_fn()

INFO:     127.0.0.1:55708 - "GET /api/v2/api?verify_key=edf4433715069a623e77a08c75617167e103a54b606fcde79540bbec4ac75d0f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:55728 - "POST /api/v2/api_call HTTP/1.1" 200 OK


TypeError: attribute name must be string, not 'NoneType'

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [241061]


WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.


INFO:     Started server process [257443]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [257443]


WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.


INFO:     Started server process [257656]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [257656]


WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.


INFO:     Started server process [257878]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [257878]


In [ ]:
# Nonblocking

for job in jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    result_job = execute_code_fn(blocking=False)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result_job.wait()

INFO:     127.0.0.1:53144 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53160 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53176 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53180 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53182 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53216 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53226 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53254 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53256 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53282 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53284 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53304 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:53310 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result = call_method(context, *queue_item.args, **queue_item.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/service.py", line 485, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 826, in execute
    ).unwrap()
      ^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 89, in unwrap
    raise self.value
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 111, in wrapper
    output = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^

SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
INFO:     127.0.0.1:53318 - "POST /api/v2/api_call HTTP/1.1" 200 OK


AssertionError: Expected 
*Your code is waiting for approval*
server_trace: 
 to be raised, but no exception was raised.

INFO:     127.0.0.1:39150 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39158 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39158 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:39158 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:39196 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39202 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39204 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39206 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39222 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39224 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39228 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39234 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39246 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:3926

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [101579]


WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.


INFO:     Started server process [106541]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
